In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(2019)

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                         shuffle=True, num_workers=2)

In [3]:
import torch.nn as nn 
import torch.nn.functional as F
from torch.autograd import Variable

import torch.optim as optim
import time

In [4]:
cnn1_channel = 64
cnn2_channel = 64
cnn3_channel = 128
cnn4_channel = 128

hidden1_size = 300
out_size = 10
epoch = 20

In [5]:
import matplotlib.pyplot as plt 

def printFilter(f):
    col = 8
    row = f.shape[0]/8
    if f.shape[0]%8:
        row+=1
    row=int(row)
    
    fig = plt.figure(figsize=(8,row))
    for i in range(1,row*col+1):
        fig.add_subplot(row,col,i)
        if i<=f.shape[0]:
            plt.imshow(f[i-1][0], cmap='gray')
    plt.show()
        

In [6]:
criterion = nn.CrossEntropyLoss()
def fit(epoch, phase, model, dataLoader):
    if phase=='train':
        vol=False
        model.train()
    else:
        vol=True
        model.eval()
        
    learning_loss=0.
    learning_correct=0
    
    for i,(data,labels) in enumerate(dataLoader):
        if phase=='train':
            opt.zero_grad()
        
        data=Variable(data,vol).cuda()
        labels=Variable(labels).cuda()
        
        outputs=model(data)
        loss = criterion(outputs, labels)
        learning_loss+=loss.item()*data.shape[0]
        
        _,preds=torch.max(outputs.data, 1)
        learning_correct+=(labels==preds).sum().item()
        
        if phase=='train':
            loss.backward()
            opt.step()
            
    epoch_loss = learning_loss /len(dataLoader.dataset)
    epoch_correct = learning_correct / len(dataLoader.dataset) * 100.
            
    print(f'phase : {phase:{5}} -> loss = {epoch_loss:{8}.{3}}, rate = {epoch_correct:{8}.{3}}%')

In [7]:
class printData(nn.Module):
    def forward(self,x):
        print(x)
        return x

class basicCNN(nn.Module):
    def __init__(self, nin, nout):
        super(basicCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(nin, nout, 3, padding = 1),
            nn.BatchNorm2d(nout),
            nn.ReLU()
        )
        
    def forward(self,x):
        #printFilter(x.clone().cpu().data)
        return self.model(x)

class myNet(nn.Module):
    def __init__(self):
        super(myNet, self).__init__()
        self.CNN = nn.Sequential(
            basicCNN(1,cnn1_channel),
            basicCNN(cnn1_channel, cnn2_channel),
            nn.MaxPool2d(2,2),
            basicCNN(cnn2_channel, cnn3_channel),
            basicCNN(cnn3_channel, cnn4_channel),
            nn.AvgPool2d(2,2)
        )
        self.FC = nn.Sequential(
            nn.Linear(cnn4_channel*7*7, hidden1_size),
            nn.Dropout(),
            nn.ReLU(),
            nn.Linear(hidden1_size,out_size),
            nn.ReLU()
        )
        
    def forward(self,x):
        x=self.CNN(x)
        x=x.view(-1,cnn4_channel*7*7)
        x=self.FC(x)
        return F.log_softmax(x, dim=1) 

In [8]:
net = myNet().cuda()
print(net)
opt = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

myNet(
  (CNN): Sequential(
    (0): basicCNN(
      (model): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (1): basicCNN(
      (model): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): basicCNN(
      (model): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (4): basicCNN(
      (model): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    

In [9]:
for e in range(epoch):
    print(f"epoch : {e}")
    t=time.time()
    fit(e,"train",net, trainloader)
    fit(e,"eval",net, testloader)
    print(f"time : {(time.time()-t):.{3}}")
    print()

epoch : 0
phase : train -> loss =    0.211, rate =     94.0%
phase : eval  -> loss =   0.0409, rate =     98.8%
time : 43.8

epoch : 1
phase : train -> loss =   0.0619, rate =     98.3%
phase : eval  -> loss =   0.0336, rate =     99.0%
time : 42.3

epoch : 2
phase : train -> loss =   0.0454, rate =     98.8%
phase : eval  -> loss =   0.0255, rate =     99.2%
time : 42.3

epoch : 3
phase : train -> loss =    0.033, rate =     99.1%
phase : eval  -> loss =   0.0205, rate =     99.3%
time : 43.2

epoch : 4
phase : train -> loss =    0.029, rate =     99.2%
phase : eval  -> loss =   0.0188, rate =     99.4%
time : 42.0

epoch : 5
phase : train -> loss =   0.0226, rate =     99.4%
phase : eval  -> loss =   0.0181, rate =     99.5%
time : 42.3

epoch : 6
phase : train -> loss =   0.0208, rate =     99.4%
phase : eval  -> loss =   0.0168, rate =     99.6%
time : 40.4

epoch : 7
phase : train -> loss =   0.0178, rate =     99.4%
phase : eval  -> loss =   0.0172, rate =     99.5%
time : 40.7

